(https://www.assemblyai.com/app)

In [1]:
import assemblyai as aai

In [ ]:
aai.settings.api_key = "e4cf6073cc75401191d3fdbd73e692d1"
transcriber = aai.Transcriber()

transcript = transcriber.transcribe("TrumpHarrisDebate.mp3")
# transcript = transcriber.transcribe("./my-local-audio-file.wav")

print(transcript.text)

In [ ]:
FILE_URL = 'TrumpHarrisDebate.mp3'
config = aai.TranscriptionConfig(speaker_labels=True)

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(
  FILE_URL,
  config=config
)

for utterance in transcript.utterances:
  print(f"Speaker {utterance.speaker}: {utterance.text}")
  #print({utterance.text})

In [ ]:
print(utterance)

In [5]:
import requests
import time

In [ ]:
# Function to upload the audio file to AssemblyAI
def upload_file(filepath):
    headers = {
        'authorization': aai.settings.api_key,
    }
    with open(filepath, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})
    return response.json()['upload_url']

# Function to request diarization
def request_diarization(upload_url):
    headers = {
        'authorization': aai.settings.api_key,
        'content-type': 'application/json'
    }
    json_data = {
        'audio_url': upload_url,
        'speaker_labels': True
    }
    response = requests.post('https://api.assemblyai.com/v2/transcript', headers=headers, json=json_data)
    return response.json()['id']

# Function to get diarization results
def get_diarization_results(transcript_id):
    headers = {
        'authorization': aai.settings.api_key,
    }
    while True:
        response = requests.get(f'https://api.assemblyai.com/v2/transcript/{transcript_id}', headers=headers)
        data = response.json()
        if data['status'] == 'completed':
            return data
        elif data['status'] == 'failed':
            raise Exception("Diarization failed.")
        time.sleep(5)

# Main execution
filepath = "TrumpHarrisDebate.mp3"
upload_url = upload_file(filepath)
transcript_id = request_diarization(upload_url)
diarization_data = get_diarization_results(transcript_id)


# Print diarization results
for word in diarization_data['words']:
    print(f"Speaker {word['speaker']} from {word['start'] // 1000:.1f}s to {word['end'] // 1000:.1f}s: {word['text']}")

In [ ]:
import requests
import time
from collections import Counter

# Your AssemblyAI API key
API_KEY = "e4cf6073cc75401191d3fdbd73e692d1"

# Function to upload the audio file to AssemblyAI
def upload_file(filepath):
    headers = {'authorization': API_KEY}
    with open(filepath, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()['upload_url']

# Function to request diarization and transcription
def request_transcription(upload_url):
    headers = {
        'authorization': API_KEY,
        'content-type': 'application/json'
    }
    json_data = {
        'audio_url': upload_url,
        'speaker_labels': True
    }
    response = requests.post('https://api.assemblyai.com/v2/transcript', headers=headers, json=json_data)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()['id']

# Function to get transcription results
def get_transcription_results(transcript_id):
    headers = {'authorization': API_KEY}
    while True:
        response = requests.get(f'https://api.assemblyai.com/v2/transcript/{transcript_id}', headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        if data['status'] == 'completed':
            return data
        elif data['status'] == 'failed':
            raise Exception("Transcription failed.")
        time.sleep(5)

# Function to analyze word frequency
def analyze_word_frequency(words):
    speakers = {}
    for word in words:
        speaker = word['speaker']
        text = word['text']
        if speaker not in speakers:
            speakers[speaker] = []
        speakers[speaker].append(text)
    
    word_counts = {speaker: Counter(" ".join(speakers[speaker]).split()) for speaker in speakers}
    return word_counts

# Main execution
filepath = "TrumpHarrisDebate.mp3"
try:
    upload_url = upload_file(filepath)
    transcript_id = request_transcription(upload_url)
    transcription_data = get_transcription_results(transcript_id)
except requests.exceptions.RequestException as e:
    print(f"HTTP error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
else:
    # Print diarization results
    for word in transcription_data['words']:
        print(f"Speaker {word['speaker']} from {word['start'] // 1000:.1f}s to {word['end'] // 1000:.1f}s: {word['text']}")
    
    # Analyze word frequency
    word_counts = analyze_word_frequency(transcription_data['words'])
    for speaker, counts in word_counts.items():
        print(f"\nWord frequency for {speaker}:")
        for word, count in counts.items():
            print(f"{word}: {count}")
    
    # Calculate total speaking time per speaker
    speaking_times = {}
    for word in transcription_data['words']:
        speaker = word['speaker']
        start_time = word['start'] / 1000
        end_time = word['end'] / 1000
        duration = end_time - start_time
        if speaker not in speaking_times:
            speaking_times[speaker] = 0
        speaking_times[speaker] += duration
    
    print("\nTotal speaking time per speaker:")
    for speaker, time_spoken in speaking_times.items():
        print(f"{speaker}: {time_spoken:.2f} seconds")

 



In [ ]:
import requests
import time

# Your AssemblyAI API key
API_KEY = "e4cf6073cc75401191d3fdbd73e692d1"

# Function to upload the audio file to AssemblyAI
def upload_file(filepath):
    headers = {'authorization': API_KEY}
    with open(filepath, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})
        response.raise_for_status()
        return response.json()['upload_url']

# Function to request diarization and transcription
def request_transcription(upload_url):
    headers = {'authorization': API_KEY}
    json_data = {'audio_url': upload_url, 'speaker_labels': True}
    response = requests.post('https://api.assemblyai.com/v2/transcript', headers=headers, json=json_data)
    response.raise_for_status()
    return response.json()['id']

# Function to get transcription results
def get_transcription_results(transcript_id):
    headers = {'authorization': API_KEY}
    while True:
        response = requests.get(f'https://api.assemblyai.com/v2/transcript/{transcript_id}', headers=headers)
        response.raise_for_status()
        data = response.json()
        if data['status'] == 'completed':
            return data
        time.sleep(5)

# Main execution
filepath = "TrumpHarrisDebate.mp3"
upload_url = upload_file(filepath)
transcript_id = request_transcription(upload_url)
transcription_data = get_transcription_results(transcript_id)

# Calculate total speaking time per speaker
speaking_times = {}
for word in transcription_data['words']:
    speaker = word['speaker']
    start_time = word['start'] / 1000
    end_time = word['end'] / 1000
    duration = end_time - start_time
    if speaker not in speaking_times:
        speaking_times[speaker] = 0
    speaking_times[speaker] += duration

# Print total speaking time per speaker
for speaker, time_spoken in speaking_times.items():
    print(f"Speaker {speaker}: {time_spoken:.2f} seconds")

# Print diarization results
for word in transcription_data['words']:
    print(f"Speaker {word['speaker']} from {word['start'] // 1000:.1f}s to {word['end'] // 1000:.1f}s: {word['text']}")


In [ ]:
# Check if assemblyai is installed 
try:
    import assemblyai 
    print("AssemblyAI is installed.") 
except ImportError: 
    print("AssemblyAI is not installed.")

In [ ]:
from collections import defaultdict, Counter 
import re 

def analyze_transcript(transcript):
    speaker_word_count = defaultdict(int)
    word_frequency = defaultdict(Counter)

    for utterance in transcript:
        speaker = utterance['speaker']
        words = re.findall(r'\b\w+\b', utterance['text'].lower())
                           
        speaker_word_count[speaker] += len(words)
        word_frequency[speaker].update(words)

    return speaker_word_count, word_frequency

speaker_word_count, word_frequency = analyze_transcript(transcript)
                           
print("Word Count per Speaker:")
for speaker, count in speaker_word_count.items():
    print(f"{speaker}: {count} words")

print("\nWord Frequency per Speaker:")
for speaker, frequency in word_frequency.items():
    print(f"{speaker}: {dict(frequency)}")


In [ ]:
from collections import defaultdict, Counter 
import re

# Function to process and analyze transcript
def analyze_transcript(transcript):
    speaker_word_count = defaultdict(int)
    word_frequency = defaultdict(Counter)

    # Assuming transcript has a "results" key containing a list of utterances
    for utterance in transcript["results"]:
        speaker = utterance['speaker']
        words = re.findall(r'\b\w+\b', utterance['text'].lower())
        
        speaker_word_count[speaker] += len(words)
        word_frequency[speaker].update(words)

    return speaker_word_count, word_frequency

# Assume transcript is obtained from the transcriber
transcript = transcriber.transcribe("TrumpHarrisDebate.mp3")

# Analyze the transcript
speaker_word_count, word_frequency = analyze_transcript(transcript)

# Print the results
print("Word Count per Speaker:", speaker_word_count)
print("Word Frequency per Speaker:", word_frequency)


In [ ]:
# Count words spoken by each speaker 

# Calculate word frequency per speaker 

In [ ]:
import assemblyai as aai
import requests
# Check if assemblyai is installed 
try:
    import assemblyai 
    print("AssemblyAI is installed.") 
except ImportError: 
    print("AssemblyAI is not installed.")

#import assemblyai
import requests

# Initialize the AssemblyAI client
#aai_client = assemblyai.Client(api_key="e4cf6073cc75401191d3fdbd73e692d1")

#import assemblyai as aai
import time

# Initialize the AssemblyAI client
aai_client = aai.Client(api_key="e4cf6073cc75401191d3fdbd73e692d1")

# Function to upload the MP3 file to AssemblyAI
def upload_audio(audio_file_path):
    headers = {'authorization': 'e4cf6073cc75401191d3fdbd73e692d1"'}
    with open(audio_file_path, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})
    return response.json()['upload_url']

# Upload the audio file
audio_file_path = "trumpHarrisDecbate.mp3"
upload_url = upload_audio(audio_file_path)

# Transcribe the uploaded audio file
transcription = aai_client.transcript.create({'audio_url': upload_url})

# Poll for the transcription result until it is complete
transcription_id = transcription['id']
transcription_result = None

while True:
    transcription_result = aai_client.transcript.get(transcription_id)
    if transcription_result['status'] == 'completed':
        break
    elif transcription_result['status'] == 'failed':
        raise Exception("Transcription failed")
    time.sleep(5)  # Poll every 5 seconds

# Get the completed transcript
transcript = transcription_result['text']




